In [1]:
import numpy as np
import itertools
import trimesh
import math
import k3d
from time import sleep

from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

# !conda install -c conda-forge pyembree
# !conda install -c conda-forge igl
# !pip install Cython
# !pip install gym

In [14]:
def compute_metrics(env, model, iter_cnt=10, max_iter=30):
    rewards, final_rewards, novp = [], [], []
    for _ in range(iter_cnt):
        state, action, mask = env.reset()
        episode_reward = 0.0
        for t in range(max_iter):
            # action =  model.act(state, mask)
            
            s = torch.FloatTensor(np.float32(state)).unsqueeze(0).cuda()
            m = torch.FloatTensor(np.float32(mask)).cuda()
            q_value = model.forward(s)
            q_value *= m
            a = torch.nn.functional.softmax(q_value)
            
            action = a.argmax().item()
            print(action)
            
            state, reward, done, info, mask = env.step(action)
            # print("REWARD: ", reward)
            # env.render(action, state)
            episode_reward += reward

            if done:
                break

        final_reward = 0
#         final_reward = env.final_reward()
#         episode_reward += 1.0 / final_reward
        rewards.append(episode_reward)
        final_rewards.append(final_reward)
        novp.append(t + 1)
    return np.mean(rewards), np.mean(final_rewards), np.mean(novp)

In [7]:
from rl.environment import *

def create_env(model_path=None):
    env = Environment(illustrate=False, 
                      model_path=model_path,
                      number_of_view_points=100)
    
    # env = CombiningObservationsWrapper(env)
    # env = StepPenaltyRewardWrapper(env, weight=1.0)
    # env = DepthMapWrapper(env)

    env = VoxelGridWrapper(env)
    env = CombiningObservationsWrapper(env)
    env = VoxelWrapper(env)
    env = StepPenaltyRewardWrapper(env)
    env = FrameStackWrapper(env, num_stack=4, lz4_compress=False)
    env = ActionMaskWrapper(env)
    return env

env = create_env(model_path="./data/Tetrahedron.obj")

### DQN

In [12]:
import torch
from rl.dqn import *

model = torch.load("./models/abc-vdqn-occl-84500.pt")
if USE_CUDA:
    model = model.cuda()

In [20]:
%%time
models_path = "./data/1kabc/simple/val/"
result = {}
for model_path in sorted(os.listdir(models_path)):
    env = create_env(os.path.join(models_path, model_path))
    result[model_path] = compute_metrics(env, model, max_iter=30, iter_cnt=10)
    print(model_path, result[model_path])

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  del sys.path[0]


75
0.3679254758198162 0.10819175215945842 0.47611722797927464
0.10819175215945842
36
0.2906894993220509 0.2524614074137004 0.5431509067357513
0.2524614074137004
60
0.37862200572490334 0.22039029997457849 0.5990123056994818
0.22039029997457849
33
0.35790689499322054 0.3793826127788002 0.7372895077720207
0.3793826127788002
83
0.3418369909104605 0.39982427851959135 0.7416612694300518
0.39982427851959135
59
0.33068849495304575 0.4528283444251926 0.7835168393782384
0.4528283444251926
85
0.32072013257670867 0.48983686224194417 0.8105569948186528
0.48983686224194417
98
0.2758499472706272 0.5512225915894764 0.8270725388601037
0.5512225915894764
13
0.362526992417014 0.5039723599145922 0.8664993523316062
0.5039723599145922
39
0.29380304323808565 0.5838685888862667 0.8776716321243523
0.5838685888862667
40
0.40880329433033696 0.5380878973795076 0.9468911917098446
0.5380878973795076
24
0.37759252749460154 0.5752086383085073 0.9528011658031088
0.5752086383085073
75
0.3679254758198162 -0.068135968047

0.32072013257670867 0.5201116497222451 0.8408317822989538
0.5201116497222451
98
0.2758248380454979 0.5788699825693535 0.8546948206148515
0.5788699825693535
13
0.362526992417014 0.5317031596913231 0.8942301521083371
0.5317031596913231
39
0.29380304323808565 0.6094765922153513 0.9032796354534369
0.6094765922153513
40
0.40880329433033696 0.5493508677901944 0.9581541621205314
0.5493508677901944
00010085_df3813ab6587454f8b8ccbc1_002.obj (-6.021852594321839, 0.0, 10.8)
75
0.4294915254237288 0.31421811608150385 0.7437096415052327
0.31421811608150385
36
0.38228813559322034 0.3999429933312954 0.7822311289245157
0.3999429933312954
60
0.36338983050847457 0.534628428231227 0.8980182587397016
0.534628428231227
33
0.271864406779661 0.6301618679921048 0.9020262747717658
0.6301618679921048
83
0.4309322033898305 0.47354341451264115 0.9044756179024717
0.47354341451264115
59
0.33322033898305087 0.5712552789194207 0.9044756179024717
0.5712552789194207
85
0.4316949152542373 0.47344870532024497 0.9051436205

0.41703389830508475 0.5213126687196701 0.9383465670247548
0.5213126687196701
64
0.4103389830508475 0.5287081911668078 0.9390471742176553
0.5287081911668078
70
0.42889830508474575 0.5101488691329095 0.9390471742176553
0.5101488691329095
65
0.36059322033898306 0.5784539538786722 0.9390471742176553
0.5784539538786722
90
0.4321186440677966 0.5073956016117924 0.939514245679589
0.5073956016117924
73
0.4306779661016949 0.5095368867707946 0.9402148528724895
0.5095368867707946
68
0.4277966101694915 0.512418242702998 0.9402148528724895
0.512418242702998
14
0.47127118644067795 0.46987780935567885 0.9411489957963568
0.46987780935567885
21
0.38754237288135596 0.5575767303414372 0.9451191032227931
0.5575767303414372
88
0.42974576271186443 0.5153733405109286 0.9451191032227931
0.5153733405109286
79
0.433135593220339 0.5129176529263214 0.9460532461466604
0.5129176529263214
95
0.3694915254237288 0.5770287921848654 0.9465203176085941
0.5770287921848654
9
0.4647457627118644 0.498589127526342 0.9633348902

0.4688598694123556 0.3336467769985825 0.802506646410938
0.3336467769985825
59
0.4635303309336459 0.4715247393284126 0.9350550702620585
0.4715247393284126
85
0.47017132652491767 0.47779677070257953 0.9479680972274972
0.47779677070257953
98
0.4672414755287683 0.4808215704264159 0.9480630459551842
0.4808215704264159
13
0.4704224566103019 0.5025171559988891 0.972939612609191
0.5025171559988891
75
0.470841006752609 -0.0222180247962937 0.4486229819563153
-0.0222180247962937
36
0.47014342318209723 0.09614337643803572 0.566286799620133
0.09614337643803572
60
0.4727384340644009 0.39307353174756493 0.8658119658119658
0.39307353174756493
33
0.4659021150733858 0.46819095235301494 0.9340930674264007
0.46819095235301494
83
0.4688598694123556 0.4653281647756786 0.9341880341880342
0.4653281647756786
59
0.4635303309336459 0.48756178682513857 0.9510921177587844
0.48756178682513857
75
0.4708689100954294 0.31041517337726743 0.7812840834726968
0.31041517337726743
36
0.47014342318209723 0.31812095549891783 

...

0.494663933846915 0.41813855364801544 0.9128024874949304
0.41813855364801544
83
0.49098876245671075 0.4219489149800879 0.9129376774367987
0.4219489149800879
59
0.4717647890310269 0.4419840280569818 0.9137488170880087
0.4419840280569818
85
0.4911301152024878 0.4226187018855209 0.9137488170880087
0.4226187018855209
98
0.4878790020496148 0.4260050049802621 0.9138840070298769
0.4260050049802621
13
0.49321506820269984 0.4206689388271771 0.9138840070298769
0.4206689388271771
39
0.4798925719132094 0.43399143511666755 0.9138840070298769
0.43399143511666755
40
0.4691851014205951 0.44483409555115017 0.9140191969717453
0.44483409555115017
24
0.49395717011802953 0.4203324067374524 0.914289576855482
0.4203324067374524
29
0.49445190472824935 0.4198376721272326 0.914289576855482
0.4198376721272326
93
0.4911654533889321 0.4231241234665499 0.914289576855482
0.4231241234665499
6
0.49441656654180505 0.4198730103136769 0.914289576855482
0.4198730103136769
74
0.4921549226093717 0.42213465424611024 0.914289

0.45517083985393847 0.46839603912695327 0.9235668789808917
0.46839603912695327
68
0.4446726656233698 0.4788942133575219 0.9235668789808917
0.4788942133575219
14
0.48043818466353677 0.44312869431735497 0.9235668789808917
0.44312869431735497
21
0.4583333333333333 0.46586004663951835 0.9241933799728517
0.46586004663951835
88
0.45119327073552423 0.47300010923732744 0.9241933799728517
0.47300010923732744
79
0.4442814293166406 0.47991195065621106 0.9241933799728517
0.47991195065621106
95
0.47121152842983827 0.4529818515430134 0.9241933799728517
0.4529818515430134
9
0.4519757433489828 0.4986350951181781 0.9506108384671609
0.4986350951181781
75
0.4448356807511737 0.04744074201305393 0.4922764227642276
0.04744074201305393
36
0.4334572248304643 0.19448992964108042 0.6279471544715447
0.19448992964108042
33
0.4354786124152321 0.21960268839777597 0.6550813008130081
0.21960268839777597
83
0.4471830985915493 0.20952421848162145 0.6567073170731708
0.20952421848162145
59
0.4422600417318727 0.2345692265

0.45461658841940533 0.3668939477792177 0.821510536198623
0.3668939477792177
00020228_f7a8b12633374114bcbd6244_000.obj (-11.343489789917973, 0.0, 18.0)
75
0.49206349206349204 -0.09567484202049975 0.3963886500429923
-0.09567484202049975
36
0.46330802852541986 -0.02478696231733729 0.43852106620808257
-0.02478696231733729
60
0.4897630549804463 0.16228624281261755 0.6520492977930639
0.16228624281261755
33
0.48090637221072 0.22072733372221204 0.7016337059329321
0.22072733372221204
83
0.4858523119392685 0.23269168347489033 0.7185439954141588
0.23269168347489033
59
0.4687140556705774 0.27247253074386796 0.7411865864144453
0.27247253074386796
85
0.5001150218541522 0.2800512148899005 0.7801662367440527
0.2800512148899005
98
0.40085116172072693 0.4094096579983903 0.8102608197191172
0.4094096579983903
13
0.47573038877386703 0.37379669635081053 0.8495270851246776
0.37379669635081053
39
0.47319990798251665 0.38119963343336183 0.8543995414158785
0.38119963343336183
40
0.38451805843110193 0.4801996257

0.48090637221072 0.24224398578450673 0.7231503579952268
0.24224398578450673
83
0.4858523119392685 0.24752648110539566 0.7333787930446641
0.24752648110539566
59
0.4687140556705774 0.30455563406689273 0.7732696897374701
0.30455563406689273
85
0.5001150218541522 0.33350243467499885 0.8336174565291511
0.33350243467499885
98
0.40085116172072693 0.44708610387763653 0.8479372655983635
0.44708610387763653
13
0.47573038877386703 0.4325546436161773 0.9082850323900443
0.4325546436161773
39
0.47319990798251665 0.43713081141741517 0.9103307193999318
0.43713081141741517
40
0.38451805843110193 0.5435419483878547 0.9280600068189567
0.5435419483878547
24
0.4917184265010352 0.46123077227155257 0.9529491987725878
0.46123077227155257
75
0.49206349206349204 0.24479579621597253 0.7368592882794646
0.24479579621597253
36
0.46330802852541986 0.3029342176384717 0.7662422461638916
0.3029342176384717
60
0.4897630549804463 0.2970472617025442 0.7868103166829905
0.2970472617025442
33
0.48090637221072 0.3160247410769

0.45221523918350387 0.5167484641305413 0.9689637033140452
0.5167484641305413
75
0.463408300005194 0.23108167603818014 0.6944899760433741
0.23108167603818014
36
0.43847712044876125 0.38020904775197 0.8186861682007313
0.38020904775197
60
0.44969615124915596 0.437392488266668 0.887088639515824
0.437392488266668
33
0.485067262244845 0.4690999297864247 0.9541671920312697
0.4690999297864247
75
0.4633823300264894 0.06791104536783238 0.5312933753943218
0.06791104536783238
36
0.43847712044876125 0.22234306882568672 0.660820189274448
0.22234306882568672
60
0.44969615124915596 0.3465183597918535 0.7962145110410095
0.3465183597918535
33
0.485067262244845 0.3695068702472686 0.8545741324921136
0.3695068702472686
83
0.443177686594297 0.4122166351722645 0.8553943217665615
0.4122166351722645
59
0.45221523918350387 0.49794248952312076 0.9501577287066246
0.49794248952312076
75
0.4633823300264894 0.28088183526695715 0.7442641652934465
0.28088183526695715
36
0.43847712044876125 0.3147235386968846 0.7532006

In [21]:
a, b, c = [], [], []
for name, res in result.items():
    a.append(res[0])
    b.append(res[1])
    c.append(res[2])
np.mean(a), np.mean(b), np.mean(c)

(-8.452115717300314, 0.0, 14.64722222222222)

### Random

In [ ]:
%%time
random_agent_func = lambda s : env.action_space.sample()
models_path = "./data/10abc/"
result = {}
for model_path in os.listdir(models_path):
    env = create_env(os.path.join(models_path, model_path))
    result[model_path] = compute_metrics(env, random_agent_func, max_iter=30)
    print(model_path, result[model_path])

00731313_58fc1a1bb4b5be10cb401503_003.obj (-2.5757095519377606, 0.25795545337154036, 4.3)
00942098_bd039b3a4a4efa75e86b7350_000.obj (-3.0503391472868215, 0.10782423963311767, 4.9)


In [37]:
df = pd.read_csv("./abc_100_train.csv")

model_paths = []
for ix, row in df.iterrows():
    if row["Greedy_novp"] < 10:
        model_paths.append(row["Name"])

In [40]:
for path in model_paths:
    try:
        os.rename(path, os.path.join("./data/1kabc/simple/train", os.path.basename(path)))
    except:
        pass